In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from lfs.helper import *
from lfs.lf_categorical import *
from lfs.lf_extration import *
from lfs.lf_numerical import *
from lfs.lf_unusable import *
from lfs.lf_context_specific import *

In [3]:
number_of_files = 1000
base_dir = 'data/labels/'
ext = '.xlsx'
df = pd.DataFrame()
for file_name in os.listdir(base_dir):
    if number_of_files < 0: break
    if not file_name.endswith(ext): continue
    number_of_files -= 1
    print('reading files', file_name)
    df = df.append(pd.read_excel(base_dir + file_name))

reading files data_for_labelling4.xlsx
reading files data_for_labelling8.xlsx
reading files data_for_labelling9.xlsx
reading files data_for_labelling5.xlsx
reading files data_for_labelling3.xlsx
reading files data_for_labelling2.csv.xlsx
reading files data_for_labelling6 (1) (1).xlsx
reading files data_for_labelling1.csv.xlsx
reading files data_for_labelling0.csv.xlsx
reading files data_for_labelling7.xlsx


In [4]:
cols = df.columns.tolist()
print(cols)

['% nans', 'Attribute_name', 'Check', 'Num of nans', 'Reason', 'Record_id', 'Total_val', 'Unnamed: 2', 'Unnamed: 9', 'check', 'max_val', 'mean', 'min_val', 'num of dist_val', 'reason', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5', 'std_dev', 'y_Arun', 'y_act', 'y_pred']


In [5]:
df['% nans'] = df['Num of nans'] / df['Total_val']

In [6]:
cols = ['y_act','Reason','Attribute_name','Total_val','num of dist_val','% nans'] + cols[10:13] + ['std_dev'] + cols[15:-4]

In [7]:
df = df.reindex(columns=cols)
df[:1]

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
0,Unusable,l,StratificationCategory2,403984,1,0.803648,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [8]:
print(df.columns.tolist())

['y_act', 'Reason', 'Attribute_name', 'Total_val', 'num of dist_val', '% nans', 'max_val', 'mean', 'min_val', 'std_dev', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5']


In [9]:
df[df.columns.tolist()[-5:]] = df[df.columns.tolist()[-5:]].astype(str)

In [10]:
df.y_act.unique()

array(['Unusable', 'Usable with extraction',
       'Usable directly categorical', 'Context_specific',
       'Usable directly numeric', 'Usable Directly Numeric',
       'Usable Directly Categorical', 'Usable with Extraction',
       'Usable Directly Categorical ', 'Usable with Extration',
       'Usable With Extraction', 'Usable with Extraction ', 'unusable',
       'Usable Directly numeric', 'Usable directly Categorical',
       'Usable Directly categorical'], dtype=object)

In [11]:
df['y_act'] = df.apply(lambda row: label_mapper(row), axis=1)

In [12]:
categories = {'Usable Directly Categorical':    1, 
              'Usable Directly Numeric':        2, 
              'Usable With Extraction':         3, 
              'Unusable':                       4, 
              'Context_Specific':               5, 
              'None':                           0
              }

In [13]:
categories

{'Context_Specific': 5,
 'None': 0,
 'Unusable': 4,
 'Usable Directly Categorical': 1,
 'Usable Directly Numeric': 2,
 'Usable With Extraction': 3}

In [14]:
df.groupby(df.y_act).size()

y_act
Context_Specific               2050
Unusable                        891
Usable Directly Categorical    2087
Usable Directly Numeric        5063
Usable With Extraction          650
dtype: int64

In [15]:
categories_size = {'Context_Specific':              2050,
                   'Unusable':                      891,
                   'Usable Directly Categorical':   2087,
                   'Usable Directly Numeric':       5063,
                   'Usable With Extraction':        650
                   }

In [16]:
catelens = [0, 2087, 5063, 650, 891, 2050]

In [17]:
find_key_word('location', df, 'Attribute_name')

[0, 7, 2, 17, 2, 0]

In [31]:
find_key_word('group', df, 'Attribute_name')

[0, 19, 2, 9, 10, 11]

In [32]:
find_key_word('lot', df, 'Attribute_name')

[0, 19, 21, 1, 8, 7]

# Usable With Extraction

|Total|Matched|Mismatched| Abstained| Accuracy | Coverage|
| --- | --- | --- | --- | --- | --- |
|   650  |  581 | 559  | 69| 58.69 | 89.85 |

In [34]:
lfs_ext = [lf_extractable_units, lf_extractable_number_sci, lf_extractable_pattern, lf_date_extraction_name, lf_extractable_name, lf_extractable_list, lf_extractable_sample_length, lf_date_extraction_samples, lf_extractable_email_url, lf_same_len_string]
len(lfs_ext)

10

In [18]:
cms = test_lfs(lfs_ext, df)

labeling function names       	       nan  category   numeric   extract  unusable   context
lf_extractable_units          	         0         1         0         6         0         2
lf_extractable_number_sci     	         0         1         1         0         0         0
lf_extractable_pattern        	         0        25         1        78        14        14
lf_date_extraction_name       	         0         6        21       149        12         1
lf_extractable_name           	         0        58        60       186        37         8
lf_extractable_list           	         0         2         3        27         5         0
lf_extractable_sample_length  	         0       123         3       274        37        57
lf_date_extraction_samples    	         0        19         0       264        17        51
lf_extractable_email_url      	         0         0         0        33        10         4


In [35]:
cm, abstained, mismatched, accuracy, coverage = test_lfs_onecategory(lfs_ext, df, 'Usable With Extraction')
cm, len(abstained), len(mismatched), accuracy, coverage

([0, 227, 86, 582, 136, 129], 68, 578, 0.568359375, 0.8953846153846153)

In [20]:
catelens

[0, 2087, 5063, 650, 891, 2050]

In [21]:
mismatched = pd.DataFrame(mismatched, columns=cols)
mismatched.to_csv('results/extractable_mismatched.csv')
mismatched.sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
64,Usable Directly Categorical,f,item,188939.0,18.0,0.001493,0.000,0.000000,0.000,0.000000,Acquisitions less disposals of valuables,nan,Changes in inventories,Collective consumption expenditure,Equals: GROSS DOMESTIC PRODUCT
411,Usable Directly Categorical,h,IMF data dissemination standard,263.0,3.0,0.304183,0.000,0.000000,0.000,0.000000,nan,Enhanced General Data Dissemination System (e-...,Special Data Dissemination Standard (SDDS),Special Data Dissemination Standard Plus (SDDS+),nan
73,Usable Directly Categorical,f,category,188939.0,18.0,0.000000,0.000,0.000000,0.000,0.000000,acquisitions_less_disposals_of_valuables,changes_in_inventories,collective_consumption_expenditure,equals_gross_domestic_product,exports_of_goods
841,Usable Directly Numeric,a,contrib_other,2720.0,634.0,0.691176,4330373.948,34795.657690,-0.149,194261.929000,1273,2091,2521,1575,2185
171,Usable Directly Categorical,h,label,9400.0,2235.0,0.000000,0.000,0.000000,0.000,0.000000,Finger_snapping Snare_drum Telephone,Flute Oboe Clarinet,Bass_drum Snare_drum Gunshot_or_gunfire,Bass_drum Snare_drum Electric_piano,Oboe Flute Clarinet
208,Usable Directly Numeric,a,Review_Total_Positive_Word_Counts,515738.0,365.0,0.000000,395.000,17.776458,0.000,21.804164,11,105,21,26,8
713,Context_Specific,f,s1p2c3plant,9597.0,118.0,0.489632,0.000,0.000000,0.000,0.000000,Jun,nan,2002-08-01 00:00:00,Jul,Aug
166,Usable Directly Categorical,h,label,9400.0,2398.0,0.000000,0.000,0.000000,0.000,0.000000,Finger_snapping Scissors Shatter,Oboe Flute Clarinet,Bass_drum Snare_drum Hi-hat,Bass_drum Electric_piano Snare_drum,Oboe Flute Saxophone
438,Unusable,l,q21CoreCompCodeReview,25090.0,2.0,0.200638,0.000,0.000000,0.000,0.000000,nan,Code Review Capability,nan,nan,nan
592,Usable Directly Categorical,h,contentType,3122.0,3.0,0.000000,0.000,0.000000,0.000,0.000000,HTML,RICH,VIDEO,HTML,HTML


In [22]:
abstained = pd.DataFrame(abstained, columns=cols)
abstained.to_csv('results/extractable_abstained.csv')
abstained.sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
383,Usable With Extraction,n,Station,513373.0,13431.0,0.506032,0.000000e+00,0.000000e+00,0.0,0.000000e+00,nan,D2-EX1504L3-05,D2-EX1504L3-07,19,201109281
748,Usable With Extraction,c,ADM1SALBNA,4861.0,17.0,0.376671,0.000000e+00,0.000000e+00,0.0,0.000000e+00,nan,National Capital Region,Cordillera Admin Region,Region I - Ilocos Region,Region II - Cagayan Valle
68,Usable With Extraction,h,currency,188939.0,378.0,0.001493,0.000000e+00,0.000000e+00,0.0,0.000000e+00,Euro,1999 ATS euro / euro,1999 ATS euro / euro,Austrian schilling,1999 BEF euro / euro
1064,Usable With Extraction,c,topic_name,2509.0,2509.0,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000e+00,Sports Fan,Latin Music,English as a Second Language,Spanish Language,Italian Language
881,Usable With Extraction,NaN,STREET,619129.0,389829.0,0.000002,0.000000e+00,0.000000e+00,0.0,0.000000e+00,570 SOUTH FOURTH STREET NO 100,1706 GREENLEAF DR,LOCAL,15623 S CENTERLINE RD,201 E PATRICK ST UNIT 249
1022,Usable With Extraction,NaN,INTRON,65188.0,1929.0,0.864960,0.000000e+00,0.000000e+00,0.0,0.000000e+00,nan,Jun-38,16/38,20/38,25/38
34,Usable With Extraction,NaN,value,72485.0,72061.0,0.000000,4.962430e+15,7.871720e+12,-152522177.6,8.707530e+13,131880000000,158614000000,152894000000,141212000000,136766000000
852,Usable With Extraction,NaN,ICO,240579.0,138846.0,0.340233,0.000000e+00,0.000000e+00,0.0,0.000000e+00,nan,% ALBERTO MOLINA MARTINEZ,% JOANNE SOUSA,% DAVID WALRATH,% BRENDAN BURKE
399,Usable With Extraction,c,National accounts base year,263.0,39.0,0.220532,0.000000e+00,0.000000e+00,0.0,0.000000e+00,2000,2002-03-01 00:00:00,2002,Original chained constant price data are resca...,nan
780,Usable With Extraction,NaN,site_info,188462.0,31.0,0.000000,0.000000e+00,0.000000e+00,0.0,0.000000e+00,Sidewalk: Curb side : Cutout,:,Sidewalk: Property side : Yard,Sidewalk: Curb side : Yard,Median : Cutout


# Usable Directly Categorical

|Total|Matched|Mismatched| Abstained| Accuracy | Coverage |
| --- | --- | --- | --- | --- | --- |
|   2087  |  1803 | 1515  | 284 | 59.27    | 86.39    |

In [82]:
def lf_same_len_num_lt(row):
    samples = get_samples(row)
    if lf_sample_cast_to_float(row):
        return 0
    sample_len = [len(sample) for sample in samples]
    sample_len_min = sum(lens<=2 for lens in sample_len)
    if len(set(sample_len)) < 2 and sample_len_min>3:
        return 3
    return 0

def lf_same_len_string_lt(row):
    samples = get_samples(row)
    if lf_cast_to_numbers(row):
        return 0
    sample_len = [len(sample) for sample in samples]
    sample_len_min = sum(lens<=10 for lens in sample_len)
    if len(set(sample_len)) < 2 and sample_len_min>3:
        return 3
    return 0

def lf_max_eq_dist(row):
    if row['num of dist_val'] == 1 or row['num of dist_val'] == 0:
        return 0
    if row['max_val'] == row['num of dist_val']:
        return 3
    return 0

In [83]:
cms = test_lfs([lf_same_len_num_lt, lf_same_len_string_lt, lf_max_lt_dist], df)

labeling function names       	       nan  category   numeric   extract  unusable   context
lf_same_len_num_lt            	         0       426       228         0       155       142
lf_same_len_string_lt         	         0       110         0        34        61        27
lf_max_lt_dist                	         0       105        75         0        22       106


In [23]:
lfs_cate = [lf_binary_category, lf_name_category, lf_coded_abbreviation, lf_finite_set_name, lf_finite_set_sample, lf_dist_string_percentage, lf_dist_num_percentage]
len(lfs_cate)

7

In [24]:
cms = test_lfs(lfs_cate, df)

labeling function names       	       nan  category   numeric   extract  unusable   context
lf_binary_category            	         0       928        46         7       101       154
lf_name_category              	         0       330       100        22        48        11
lf_coded_abbreviation         	         0       103         0         0        27        11
lf_finite_set_name            	         0       396        13        63       100        27
lf_finite_set_sample          	         0       226         1        83        64        79
lf_dist_string_percentage     	         0       808       161       210       100       185
lf_dist_num_percentage        	         0       515       143         0        17       265


In [27]:
cm, abstained, mismatched, accuracy, coverage = test_lfs_onecategory(lfs_cate, df, 'Usable Directly Categorical')
cm, len(abstained), len(mismatched), accuracy, coverage

([0, 1803, 419, 291, 276, 529],
 284,
 1515,
 0.5927021696252466,
 0.8639195016770483)

In [28]:
catelens

[0, 2087, 5063, 650, 891, 2050]

In [29]:
mismatched = pd.DataFrame(mismatched, columns=cols)
mismatched.to_csv('results/cate_mismatched.csv')
mismatched.sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
62,Usable With Extraction,NaN,sna93_table_code,188939.0,161.0,0.000000,0.0,0.000000,0.00,0.000000,1.1,Footnote,Excluding works of arts and antiques.,Includes a statistical discrepancy.,Includes Acquisitions less disposals of valuab...
581,Usable Directly Numeric,a,UnitPrice,541909.0,1630.0,0.000000,38970.0,4.611114,-11062.06,96.759764,2.55,3.39,2.75,7.65,4.25
338,Usable With Extraction,NaN,job_description,22000.0,18744.0,0.000000,0.0,0.000000,0.00,0.000000,TeamSoft is seeing an IT Support Specialist to...,The Wisconsin State Journal is seeking a flexi...,Report this job About the Job DePuy Synthes Co...,Why Join Altec? If you阵朐諍擘阵雮阵隷阵雮諍霂re consideri...,Position ID#阵朐阵擘阵雮阵隷 76162 # Positions阵朐阵擘阵雮阵隷...
406,Unusable,l,q17HirChaHardAssessSkills,25090.0,2.0,0.123515,0.0,0.000000,0.00,0.000000,nan,nan,Hard to assess skills before onsite,Hard to assess skills before onsite,nan
360,Unusable,l,n.status,2031.0,0.0,1.000000,0.0,0.000000,0.00,0.000000,nan,nan,nan,nan,nan
530,Usable With Extraction,b,value,19998.0,10.0,0.548805,0.0,0.000000,0.00,0.000000,nan,4 of 5 bubbles,1 of 5 bubbles,3 of 5 bubbles,5 of 5 bubbles
820,Context_Specific,a,farmwork3,9597.0,3.0,0.145462,2.0,1.245946,0.00,0.520161,1,2,nan,False,1
786,Unusable,l,BoroCode,324403.0,1.0,0.000000,4.0,4.000000,4.00,0.000000,4,4,4,4,4
289,Unusable,l,n.company_type,59228.0,0.0,1.000000,0.0,0.000000,0.00,0.000000,nan,nan,nan,nan,nan
634,Context_Specific,o,DESCRICAO_NACIONALIDADE,21124.0,4.0,0.000000,0.0,0.000000,0.00,0.000000,BRASILEIRA NATA,BRASILEIRA (NATURALIZADA),PORTUGUESA COM IGUALDADE DE DIREITOS,ESTRANGEIRO,ESTRANGEIRO


In [39]:
abstained = pd.DataFrame(abstained, columns=cols)
abstained.to_csv('results/cate_abstained.csv')
abstained.sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
1055,Usable Directly Categorical,NaN,YEAR,1224.0,24.0,0.000000,2015.0,2003.500000,1992.0,6.922187,1992,1993,1994,1995,1996
257,Usable Directly Categorical,i,fastestLap,23777.0,77.0,0.773605,78.0,41.061676,2.0,17.154842,39,41,58,43,50
403,Usable Directly Categorical,NaN,Date,8190.0,182.0,0.000000,0.0,0.000000,0.0,0.000000,2010-05-02 00:00:00,2010-12-02 00:00:00,19/02/10,26/02/10,2010-05-03 00:00:00
245,Usable Directly Categorical,j,Requisition Number,346018.0,5997.0,0.958473,0.0,0.000000,0.0,0.000000,REQ0011118,REQ0011932,REQ0011476,nan,REQ0013911
900,Usable Directly Categorical,g,Police District,64618.0,102.0,0.000573,0.0,0.000000,0.0,0.000000,590,822,830,850,871
160,Usable Directly Categorical,i,fiscal_year,68023.0,67.0,0.002896,2016.0,2002.546811,1950.0,11.183397,2015,2014,2013,2012,2011
885,Usable Directly Categorical,NaN,GROUP,619129.0,2589.0,0.000000,9707.0,865.599382,0.0,1970.373963,1736,0,928,557,1961
276,Usable Directly Categorical,f,Con,1518.0,158.0,0.000000,0.0,0.000000,0.0,0.000000,Afghanistan,Afghanistan/Pakistan,Algeria,Angola,Angola/Burundi/Central African Republic/Democr...
860,Usable Directly Categorical,g,Police District,82958.0,99.0,0.000241,0.0,0.000000,0.0,0.000000,690,590,822,642,620
36,Usable Directly Categorical,j,employer_phone,374362.0,49220.0,0.361428,0.0,0.000000,0.0,0.000000,nan,2625444811,662-254-7200,720-859-6111,206 391 7466


# Usable Directly Numeric

|Total|Matched|Mismatched| Abstained| Accuracy | Coverage|
| --- | --- | --- | --- | --- | --- |
|   5063  |  5055 | 3459  | 8| 63.84 | 99.84|

In [17]:
lfs_num = [lf_cast_to_numbers, lf_has_mean_std, lf_has_mean_dev_min,lf_max_min_difference,lf_sample_cast_to_float, lf_std_gt_mean]
len(lfs_num)

6

In [18]:
cms = test_lfs(lfs_num, df)

labeling function names       	       nan  category   numeric   extract  unusable   context
lf_cast_to_numbers            	         0      1092      5055        24       596      1747
lf_has_mean_std               	         0       965      5002        21       171      1663
lf_has_mean_dev_min           	         0       364      2496        21       142       673
lf_max_min_difference         	         0        14       958        13        86       400
lf_sample_cast_to_float       	         0         0      1865         2        20       346
lf_std_gt_mean                	         0       581      3435         8        23      1255


In [19]:
cm, abstained, mismatched, accuracy, coverage = test_lfs_onecategory(lfs_num, df, 'Usable Directly Numeric')
cm, len(abstained), len(mismatched), accuracy, coverage

([0, 1106, 5056, 24, 596, 1748],
 7,
 3474,
 0.6372573733299722,
 0.9986174205016789)

In [43]:
catelens

[0, 2087, 5063, 650, 891, 2050]

In [44]:
mismatched = pd.DataFrame(mismatched, columns=cols)
mismatched.to_csv('results/num_mismatched.csv')
mismatched.sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
44,Context_Specific,NaN,1995,264.0,192.0,0.068182,2.274911e+01,1.929149e+00,0.000000,3.785024,2.138784453,0,0.016472597,4.424155104,0.197623255
232,Context_Specific,a,lvs5graze2,9597.0,18.0,0.380640,1.200000e+04,4.529727e+00,0.000000,155.666517,1,0,5,7,12
1032,Unusable,l,headcount,5807.0,1.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0,0,0,0,0
90,Context_Specific,n,vx,63992.0,63971.0,0.000000,1.582761e+00,-6.970000e-05,-1.565658,0.407332,0.45098439,-0.95970893,-0.11458433,0.18220614,-0.064055555
912,Context_Specific,a,farmwork27,9597.0,2.0,0.997499,2.000000e+00,1.666667e+00,1.000000,0.471405,nan,2,1,nan,2
605,Unusable,l,Unnamed: 0,2799914.0,2799914.0,0.000000,2.799913e+06,1.399956e+06,0.000000,808265.550800,0,1,2,3,4
32,Usable Directly Categorical,h,outcome_hour,29421.0,22.0,0.000000,2.300000e+01,1.400578e+01,0.000000,3.863203,16,18,17,14,19
199,Usable Directly Categorical,i,q12JobCritCompensation,25090.0,2.0,0.000000,1.000000e+00,3.336389e-01,0.000000,0.471512,1,0,0,0,0
338,Context_Specific,a,othrhrcd1,9597.0,10.0,0.480671,3.520000e+02,1.031300e-01,0.000000,5.090769,0,nan,10,7,35
621,Context_Specific,a,s1p1c2,9597.0,49.0,0.465145,5.600000e+01,2.477518e+01,2.000000,15.341242,14,12,41,26,33


In [45]:
abstained = pd.DataFrame(abstained, columns=cols)
abstained.to_csv('results/num_abstained.csv')
abstained

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
372,Usable Directly Numeric,a,LOCATION,310579.0,285.0,0.059885,0.0,0.000000,0.0,0.000000,nan,"(41.97562073966174, -87.72838313514907)","(41.90999558160536, -87.72639469321471)","(41.95302475075328, -87.78668323766634)","(41.83773267497158, -87.66589444999927)"
574,Usable Directly Numeric,a,distance,258.0,126.0,0.000000,0.0,0.000000,0.0,0.000000,28,12,"11,2","12,9","18,5"
577,Usable Directly Numeric,a,temp_inside,258.0,11.0,0.034884,0.0,0.000000,0.0,0.000000,"21,5","22,5",20,nan,21
966,Usable Directly Numeric,a,average_review_rating,10000.0,19.0,0.001800,0.0,0.000000,0.0,0.000000,4.9 out of 5 stars,4.5 out of 5 stars,3.9 out of 5 stars,5.0 out of 5 stars,4.7 out of 5 stars
96,Usable Directly Numeric,NaN,Main Source of Income,41544.0,3.0,0.000000,0.0,0.000000,0.0,0.000000,Wage/Salaries,Other sources of Income,Enterpreneurial Activities,Wage/Salaries,Wage/Salaries
567,Usable Directly Numeric,NaN,install_size_kw_buckets,48722.0,48530.0,0.001190,0.0,0.000000,0.0,0.000000,[[0171][5231][10114][1545][2016][252][352][451...,[[060][550][1024][1518][2023][2515][3012][3515...,[[0364][5313][1057][1513][207][258][305][351][...,[[0320][567][1020][1512][251][302][351][401][4...,[[0125][5219][10227][15124][2039][2520][3015][...
774,Usable Directly Numeric,NaN,location,188462.0,160187.0,0.013398,0.0,0.000000,0.0,0.000000,(37.7216893346731 -122.430482526944),(37.8042860176436 -122.435150585742),(37.7647299507493 -122.446671573844),(37.7697365039652 -122.417760336994),(37.7583327223052 -122.412354747671)
892,Usable Directly Numeric,a,educ37,9597.0,2.0,0.999687,2.0,0.666667,0.0,0.942809,nan,False,2,False,nan


# Unusable

|Total|Matched|Mismatched| Abstained| accuracy | coverage|
| --- | --- | --- | --- | --- | --- |
|   891  |  856 | 414  | 70 | 66.48 | 92.14|

In [17]:
lfs_unusable = [lf_unusable, lf_unusable_nans]

In [18]:
cms = test_lfs(lfs_unusable, df)

labeling function names       	       nan  category   numeric   extract  unusable   context
lf_unusable                   	         0         7       148        70       751       146
lf_unusable_nans              	         0        30         2         0       337        12


In [19]:
cm, abstained, mismatched, accuracy, coverage = test_lfs_onecategory(lfs_unusable, df, 'Unusable')
cm, len(abstained), len(mismatched), accuracy, coverage

([0, 37, 150, 70, 821, 157], 70, 414, 0.6647773279352227, 0.9214365881032548)

In [20]:
catelens

[0, 2087, 5063, 650, 891, 2050]

In [42]:
mismatched = pd.DataFrame(mismatched, columns=cols)
mismatched.to_csv('results/unusable_mismatched.csv')
mismatched.sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
251,Usable Directly Categorical,e,q26FrameLearnExpressJS,25090,2,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,Will Learn,NaN,Know,NaN,NaN
515,Context_Specific,a,farmwork23,9597,2,99.301865,2.000000e+00,1.507463e+00,1.000000,4.999443e-01,NaN,2,1,NaN,2
247,Usable Directly Categorical,i,q23FrameBackboneDotJS,25090,3,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,NaN,0,1,NaN,NaN
208,Usable Directly Numeric,NaN,yearly_sunlight_kwh_e,52,52,NaN,2.585561e+10,2.580733e+09,0.000000,4.228606e+09,0,2420185385,4.49072e+07,6054636897,1020895160
0,Usable Directly Categorical,e,ji_live_in_dom_svc_contract,374362,3,99.359444,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,NaN,A,Y,N,N
238,Usable Directly Numeric,NaN,SUPPORT_SERVICES_EXPENDITURE,1224,1224,NaN,2.350185e+07,2.631397e+06,139963.000000,3.280875e+06,735036,350902,1007732,483488,8520926
194,Usable Directly Numeric,NaN,lng_avg,52,52,NaN,-6.988566e+01,-9.364726e+01,-157.507901,1.899155e+01,-116.635,-86.8507,-148.508,-111.895,-92.6524
24,Context_Specific,a,s1p2c5,9597,24,99.322705,5.600000e+01,3.413846e+01,4.000000,1.729907e+01,NaN,33,29,14,12
325,Context_Specific,n,z,63984,63959,NaN,2.184194e+01,1.574555e-03,-19.199339,8.976619e-01,0.0671728,0.108988,-0.0105302,-0.0895059,-0.284885
206,Usable Directly Numeric,NaN,yearly_sunlight_kwh_n,52,52,NaN,1.215200e+10,1.055046e+09,4745.559338,2.273631e+09,4745.56,1128360422,2.18659e+06,1780582587,455410026


In [43]:
abstained = pd.DataFrame(abstained, columns=cols)
abstained.to_csv('results/unusable_abstained.csv')
abstained.sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
32,Unusable,l,q23FramePadrino,25090,2,NaN,0.0,0.000000,0.0,0.000000,NaN,NaN,Padrino,Padrino,Padrino
16,Unusable,l,ZoneDist4,324403,6,NaN,0.0,0.000000,0.0,0.000000,NaN,R5,M3-1,R7B,R2
7,Unusable,k,ad7521,9597,7,23.997082,0.0,0.000000,0.0,0.000000,0,1,0,1,intercropping
10,Unusable,k,isbn,10000,9300,7.000000,0.0,0.000000,0.0,0.000000,439023483,439554934,316015849,61120081,743273567
27,Unusable,l,q22LangProfPython,25090,2,NaN,0.0,0.000000,0.0,0.000000,NaN,NaN,Python,Python,Python
30,Unusable,l,q23FrameStruts,25090,2,NaN,0.0,0.000000,0.0,0.000000,NaN,NaN,Struts,Struts,Struts
14,Unusable,l,ZoneDist4,89830,2,NaN,0.0,0.000000,0.0,0.000000,NaN,R5,R6A,R5,R6A
25,Unusable,l,q21CoreCompDatabaseDesign,25090,2,NaN,0.0,0.000000,0.0,0.000000,NaN,NaN,Database Design,Database Design,Database Design
24,Unusable,l,q17HirChaNoDiversCandidates,25090,2,NaN,0.0,0.000000,0.0,0.000000,NaN,Not enough diverse candidates,NaN,Not enough diverse candidates,Not enough diverse candidates
3,Unusable,k,Sentence_Index,662207,27,0.000000,26.0,3.697087,0.0,3.129019,0,1,2,3,4


# Context_Specific

In [29]:
def lf_unusable(row):
    if lf_unusable_dist(row) or lf_unusable_nans(row):
        return 4
    return 0  

def lf_unusable_dist(row):
    # Case k. A number indicating the position of a record in its dataset table, e.g., serial number
    # Case l. An attribute that is likely the primary key in its dataset table, or an attribute whose values will almost surely be unique for all records in its dataset table but is not a numeric feature
    # PRIORITY: 1
    # 1. # distinct == 1
    # 2. % distinct == 100%
    # 3. % nans     > 99%
    # [TODO] total_nans seems to be a loose contrain here
    # [51, 191, 80, 786, 219, 0]
    total_val = row['Total_val']
    dist_val = row['num of dist_val']
    total_nans = row['% nans']
    if dist_val <= 1 or (dist_val/total_val) > 0.99 or total_nans > 99:
        return 4
    return 0


def lf_unusable_nans(row):
    # Case k. A number indicating the position of a record in its dataset table, e.g., serial number
    # Case l. An attribute that is likely the primary key in its dataset table, or an attribute whose values will almost surely be unique for all records in its dataset table but is not a numeric feature
    # PRIORITY: 1
    # 1. samples are of NaN values
    # [TODO] % nans should be more precise here, while some of them are not available
    # [30, 2, 0, 337, 12, 0]
    def check_nan(string):
        return string == 'nan'

    samples = get_samples(row)
    nans_in_samples = [check_nan(sample) for sample in samples]
    if sum(nans_in_samples) >= 3:
        return 4
    return 0

In [158]:
from nltk.corpus import words

In [206]:
import re

def lf_context_specific(row):
    if lf_unusable(row):
        return 0
    if lf_key_word_id(row, False):
        return 5
    return 0
    
def lf_key_word_id(row, single=True):
    # a key word in attribute name while has different representation from classes
    # PRIORITY: 2
    # e.g. student_id  studentID
    # 1. attribute name contains id
    
    if single and lf_unusable(row):
        return 0
    
    string = str(row['Attribute_name'])
    parts = re.split('[^a-zA-Z0-9]', string)
    for part in parts:
        if part.lower() == 'id':
            return 5
    if re.findall('ID', string):
        return 5
    return 0

def lf_undestandable_name(row, single=True):
    # if the attribute name is not human understandable, mark it as context specfic
    # PRIORITY: 2
    # e.g. CB2010, x, xy
    # 1. split the attribute name into individual parts
    
    if single and lf_unusable(row):
        return 0
    
    string = str(row['Attribute_name']).lower()
    first_name = re.search('[A-Z]?[a-z]*', 'n.programs').group()
    if first_name == '1':
        return 5
    if first_name.lower() not in words.words():
        return 5
    return 0

def lf_ambiguous_terms(row, single=True):
    # if the attribute name is a single ambiguous term
    # PRIORITY: 2
    # e.g. name, id
    
    if single and lf_unusable(row):
        return 0
    
    if str(row['Attribute_name']).lower() == 'id' or str(row['Attribute_name']).lower() == 'name':
        return 5
    if str(row['Attribute_name']).lower()[-2:] == 'id' or str(row['Attribute_name']).lower()[-4:] == 'name':
        return 5
    return 0

In [207]:
re.search('[A-Z]?[a-z]*', 'n.programs').group()

'n'

In [209]:
lfs_cs = [lf_undestandable_name]

In [210]:
cms = test_lfs(lfs_cs, df)

KeyboardInterrupt: 

In [44]:
df[(df.y_act=='Context_Specific')].sample(10)

,y_act,Reason,Attribute_name,Total_val,num of dist_val,% nans,max_val,mean,min_val,std_dev,sample_1,sample_2,sample_3,sample_4,sample_5
211,Context_Specific,a,lvs3graze1,9597,12,42.752944,12.0,0.396250,0.0,2.016422,0,NaN,12,3,9
165,Context_Specific,NaN,Team,2150,297,NaN,1463.0,1291.615814,1102.0,103.268764,1207,1210,1228,1260,1374
794,Context_Specific,a,s2p1c3qharv,9597,204,63.665729,8000000.0,7353.813651,0.0,172758.422900,NaN,100,90,105,200
916,Context_Specific,a,s2p2c6sold,9597,5,73.887673,300.0,0.323224,0.0,8.376567,NaN,0,200,300,50
631,Context_Specific,o,CODIGO_COR_RACA,21124,5,0.000000,5.0,1.813482,1.0,0.956523,3,1,2,4,5
1042,Context_Specific,NaN,ENROLL,359836,23399,NaN,1077381.0,3123.834339,0.0,14008.465000,7568,19961,2045,2972,3539
1063,Context_Specific,a,hm15owner,9597,4,68.219235,3.0,0.236393,0.0,0.486729,NaN,3,1,2,0
13,Context_Specific,NaN,ADM1DHS,4861,28,NaN,9999.0,1945.311664,1.0,3948.520276,8,13,70,23,68
728,Context_Specific,a,s1p2c4area,9597,41,98.582890,70.0,10.446544,0.0,9.809756,5,15,NaN,14,12
1061,Context_Specific,a,hm15,9597,14,58.518287,1000.0,0.576488,0.0,16.074431,0,NaN,10,1,3
